In [209]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [210]:
file=pd.read_csv("/content/crack_detection - crack_detection.csv")

In [211]:

file.drop('Unnamed: 0', axis=1, inplace=True)


In [212]:
file.head()

,mean_r,std_r,kur_r,skew_r,mean_g,std_g,kur_g,skew_g,mean_b,std_b,...,contrast_r,contrast_g,contrast_b,corr_rg,corr_rb,corr_gb,moment_r,moment_g,moment_b,label
0,107.5734,29.267480,0.744029,0.408828,110.2533,26.953218,1.254689,-0.135613,113.5715,25.422051,...,9.181953,177,173,162,0.951368,0.913876,0.803333,856.585412,726.475939,0
1,178.6713,14.917756,3.192406,0.238175,184.4947,14.882156,3.035196,0.141160,190.8538,14.806283,...,9.207314,142,136,129,0.988325,0.989474,0.976787,222.539456,221.478572,0
2,176.6517,34.618255,-0.146336,-0.421552,161.3083,33.688699,0.057178,-0.521476,147.2760,30.809486,...,9.186737,199,200,194,0.968581,0.967398,0.890845,1198.423587,1134.928451,1
3,113.5760,20.084502,0.719752,0.737728,114.0056,19.536222,0.655846,0.729521,113.6179,18.108592,...,9.197973,146,142,138,0.987383,0.983663,0.961305,403.387224,381.663969,0
4,125.3565,29.251595,0.430344,0.833496,129.5277,28.661968,0.475502,0.804161,131.9745,27.475848,...,9.189257,173,176,177,0.980727,0.986702,0.950291,855.655808,821.508433,1


In [213]:
file.shape

(700, 24)

In [214]:
x=file.iloc[:,:-1].values
y=file.iloc[:,-1].values

In [215]:
x.shape,y.shape

((700, 23), (700,))

In [225]:
train_x, test_x,val_x = x[:420], x[420:420+140],x[420+140:420+140+140]
train_y, test_y ,val_y= y[:420], y[420:420+140],y[420+140:420+140+140]



In [226]:
test_x.shape

(140, 23)

In [283]:
norm = np.linalg.norm(train_x,axis=1)[:,np.newaxis]
X = train_x/norm
norm = np.linalg.norm(test_x,axis=1)[:,np.newaxis]
X_test= test_x/norm
norm = np.linalg.norm(val_x,axis=1)[:,np.newaxis]
X_val= val_x/norm

In [284]:
def sigmoid(X):
  return 1/(1+np.exp(-X))

In [285]:
def predicted(X,m,c):
  y=np.dot(X,m)+c
  Y=sigmoid(y)
  return Y

In [286]:
def loss(Y,train_y):
  return np.mean(-train_y*np.log(Y)-(1-train_y)*np.log(1-Y))

In [287]:
def coefficient(X,train_y):
  alpha=0.009
  m=np.random.randn((23))
  c=np.random.randn((1))
  losses=[]
  for i in range(10000):

    Y_pred=predicted(X,m,c)
    mean_=np.mean(np.dot((Y_pred-train_y),X))
    mean=np.mean(Y_pred-train_y)
    c=c-(alpha)*mean
    m=m-(alpha)*mean_
    los=loss(Y_pred,train_y)
    losses.append(los)
    if i%1000==0:
      print(los)

  return m,c

In [288]:
m,c=coefficient(X,train_y)
print(m,c)

0.9693920518293107
0.743135486758791
0.7223889861406206
0.7037767527790031
0.6870853259857895
0.6721153461146676
0.6586832410045674
0.6466218808829919
0.635780463636394
0.626023866462248
[-0.83325972 -0.52603763 -1.6031965  -2.2667246  -1.01458781 -1.61060339
 -0.93295284  0.98045062 -0.97025024 -2.20851101 -2.23581878 -0.75519021
 -0.21660218 -0.5965202  -0.12349576 -1.98534874  1.10650275 -2.22986528
 -1.41076354 -0.73740238  0.58610225 -0.56544365 -1.83511735] [2.36155957]


In [289]:
y_pred=predicted(X_test,m,c)
y_pred=np.where(y_pred>0.5,1,0)
total=np.sum(test_y==y_pred)
print(total/y_pred.shape[0])

0.7714285714285715


In [290]:
y_pred_train=predicted(X,m,c)
y_pred_train = np.where(y_pred_train>0.5,1,0)
y_pred_val=predicted(X_val,m,c)
y_pred_val = np.where(y_pred_val>0.5,1,0)
y_pred_test=predicted(X_test,m,c)
y_pred_test = np.where(y_pred_test>0.5,1,0)

In [310]:
def function(y_pred_train,train_y):
  positive_true=0
  negative_true=0
  negative_false=0
  positive_false=0
  for i in range(len(y_pred_train)):
    if (y_pred_train[i]==1 and train_y[i]==1):
      positive_true+=1
    elif y_pred_train[i]==1 and train_y[i]==0:
      positive_false+=1
    elif y_pred_train[i]==0 and train_y[i]==1:
        negative_false+=1
    elif y_pred_train[i]==0 and train_y[i]==0:
        negative_true+=1
  precision=positive_true/(positive_true+positive_false)
  recall=positive_true/(positive_true+negative_false)
  print(f"precision is ={precision}")
  print(f"recall is ={recall}")
  f1=2*(precision*recall/(recall+precision))
  print(f"f1 value is ={f1}")
  # print(positive_true,negative_true,negative_false,positive_false)

In [311]:

function(y_pred_train,train_y)

precision is =0.7746478873239436
recall is =0.7746478873239436
f1 value is =0.7746478873239436


In [312]:
function(y_pred_test,test_y)

precision is =0.7746478873239436
recall is =0.7746478873239436
f1 value is =0.7746478873239436


In [313]:

function(y_pred_val,val_y)

precision is =0.7
recall is =0.7538461538461538
f1 value is =0.725925925925926
